In [98]:
import requests
import pandas as pd
## FOOD INVESTIMENT
# URL da API
api_url_invest = "https://olinda.bcb.gov.br/olinda/servico/SICOR/versao/v2/odata/InvestRegiaoUFProduto?%24format=text%2Fcsv"

# Faz a solicitação GET à API
response = requests.get(api_url_invest)

# Verifica se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Salva o conteúdo da resposta em um arquivo temporário
    with open('temp.csv', 'w', encoding='utf-8') as temp_file:
        temp_file.write(response.text)
    
    # Lê o arquivo temporário como um DataFrame do Pandas, especificando a codificação
    df_invest = pd.read_csv('temp.csv', sep=',', encoding='utf-8')
    
    # Exibe as primeiras linhas do DataFrame
    print(df_invest.head())
    
    # Remove o arquivo temporário
    import os
    os.remove('temp.csv')
else:
    print("Falha na solicitação à API")

                                         nomeProduto    nomeRegiao nomeUF  \
0                                           "TRATOR"       SUDESTE     RJ   
1                                 "PROTEÇÃO DO SOLO"      NORDESTE     CE   
2                                          "BOVINOS"  CENTRO-OESTE     GO   
3  "SISTEMAS DE CAPTAÇÃO, RETENÇÃO E APROVEITAMEN...      NORDESTE     BA   
4                                          "BOVINOS"         NORTE     PA   

   MesEmissao  AnoEmissao  cdPrograma  cdSubPrograma  cdFonteRecurso  \
0           6        2017         154             30             505   
1           3        2023           1             56             502   
2           6        2014           1              0             505   
3           6        2014           1              0             502   
4          11        2020           1              2             501   

   cdTipoSeguro  QtdCusteio   VlCusteio  Atividade  cdModalidade  
0             3           3      3991

In [99]:
## Tratamento DF investimentos

df_invest["VlCusteio"] = df_invest["VlCusteio"].apply(lambda x: x.split(',')[0])
df_invest["VlCusteio"] = df_invest["VlCusteio"].astype(int) 
df_invest["nomeProduto"] = df_invest["nomeProduto"].str.replace('"','')
df_invest = df_invest[["nomeRegiao","nomeUF","MesEmissao","AnoEmissao","VlCusteio","nomeProduto"]]
df_invest['Key1'] = df_invest["nomeUF"].astype(str) + df_invest["MesEmissao"].astype(str) + df_invest["AnoEmissao"].astype(str) + df_invest["nomeProduto"].astype(str)
df_invest = df_invest[["nomeRegiao","nomeUF","MesEmissao","AnoEmissao","VlCusteio","nomeProduto","Key1"]]
df_invest = df_invest.groupby("Key1").agg({
    "nomeRegiao":"first",
    "nomeUF":"first",
    "MesEmissao":"first",
    "AnoEmissao":"first",
    "VlCusteio":"sum",
    "nomeProduto":"first"
}).reset_index()
#df_invest = df_invest[["nomeRegiao","nomeUF","MesEmissao","AnoEmissao","VlCusteio","nomeProduto"]].reset_index()
df_invest = df_invest[["nomeRegiao","nomeUF","MesEmissao","AnoEmissao","VlCusteio","nomeProduto"]]

In [100]:
import pandas_gbq #Conexão com BigQuery pelo Pandas
from google.oauth2 import service_account #Conexão com Google Cloud

In [101]:
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform'
]

credentials = service_account.Credentials.from_service_account_info(
{
  "type": "service_account",
  "project_id": "fluted-current-384614",
  "private_key_id": "02006c_____________21ff4",
  "private_key": "MIIEuwIBADANBgkq______________NY6ptYftqoI________________ToRWbnd5PpebzJRV1q_______ZBJFrasUDQxQ/E_______jLC38t/Ef+3h______6eWgywdjckc_______eID046N9EQe____________mBRu17sU_______r\nZee3087NDfikmy____",
  "client_email": "enterprise-challenge@fluted-current-384614.iam.gserviceaccount.com",
  "client_id": "10___________1449",
  "auth_uri": "https://accounts.google.com/o/oauth2/auth",
  "token_uri": "https://oauth2.googleapis.com/token",
  "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
  "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/enterprise-challenge%40fluted-current-384614.iam.gserviceaccount.com",
  "universe_domain": "googleapis.com"
}
)

pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "fluted-current-384614"

In [132]:
pandas_gbq.to_gbq(df_invest, 'EC.Dados_Invest', project_id='fluted-current-384614', if_exists='replace')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


In [55]:
import requests
import pandas as pd

## Clima Tempo
# URL da API
api_url_clite = "http://apiadvisor.climatempo.com.br/api/v1/anl/synoptic/locale/BR?token=1646d722a05b18829ffbbe9ce324d1fc"

# Faz a solicitação GET à API
response_clite = requests.get(api_url_clite)

# Verifica se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Salva o conteúdo da resposta em um arquivo temporário
    with open('temp.csv', 'w', encoding='utf-8') as temp_file:
        temp_file.write(response.text)
    
    # Lê o arquivo temporário como um DataFrame do Pandas, especificando a codificação
    df_clite = pd.read_csv('temp.csv', sep=',', encoding='utf-8')
    
    # Exibe as primeiras linhas do DataFrame
    print(df_clite.head())
    
    # Remove o arquivo temporário
    import os
    os.remove('temp.csv')
else:
    print("Falha na solicitação à API")

Empty DataFrame
Columns: [{"location":{"name":"Brasilia", region:"Distrito Federal", country:"Brazil", lat:-15.78, lon:-47.92, tz_id:"America/Sao_Paulo", localtime_epoch:1692476389, localtime:"2023-08-19 17:19"}, current:{"last_updated_epoch":1692476100, last_updated:"2023-08-19 17:15", temp_c:27.0, temp_f:80.6, is_day:1, condition:{"text":"Partly cloudy", icon:"//cdn.weatherapi.com/weather/64x64/day/116.png", code:1003}, wind_mph:9.4, wind_kph:15.1, wind_degree:220, wind_dir:"SW", pressure_mb:1016.0, pressure_in:30.0, precip_mm:0.0, precip_in:0.0, humidity:34, cloud:75, feelslike_c:25.6, feelslike_f:78.2, vis_km:10.0, vis_miles:6.0, uv:7.0, gust_mph:9.6, gust_kph:15.5}}]
Index: []

[0 rows x 33 columns]


In [127]:
caminho_arquivo_custoprod = "https://portaldeinformacoes.conab.gov.br/downloads/arquivos/CustoProducao.txt"
df_custoprod = pd.read_csv(caminho_arquivo_custoprod, delimiter=';')
df_custoprod = df_custoprod[df_custoprod["safra"].astype(str)=="TODAS                                   "]
df_custoprod = df_custoprod[["empreendimento","ano","mes","produto","uf","vlr_custo_variavel_ha","vlr_custo_fixo_ha"]]
df_custoprod

,empreendimento,ano,mes,produto,uf,vlr_custo_variavel_ha,vlr_custo_fixo_ha
0,AGRICULTURA EMPRESARIAL,2018,1,ALGODAO EM PLUMA,BA,6221.64,624.87
1,AGRICULTURA EMPRESARIAL,2018,1,ALGODAO EM PLUMA,GO,5493.66,365.92
2,AGRICULTURA EMPRESARIAL,2018,1,ALGODAO EM PLUMA,MS,5192.27,577.95
3,AGRICULTURA EMPRESARIAL,2018,1,ALGODAO EM PLUMA,MT,6662.14,544.55
4,AGRICULTURA EMPRESARIAL,2018,1,ALGODAO EM PLUMA,MT,5802.43,661.74
...,...,...,...,...,...,...,...
2868,SOCIOBIODIVERSIDADE,2022,8,UMBU,BA,3504.83,11.05
2869,SOCIOBIODIVERSIDADE,2022,8,UMBU,MG,16971.58,22.10
2870,SOCIOBIODIVERSIDADE,2022,8,UMBU,PB,2905.83,64.00
2871,SOCIOBIODIVERSIDADE,2022,8,UMBU,PB,24340.44,66.31


In [130]:
pandas_gbq.to_gbq(df_custoprod, 'EC.Dados_Custo_Prod', project_id='fluted-current-384614', if_exists='replace')

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]


In [149]:
caminho_arquivo_levgraos = "https://portaldeinformacoes.conab.gov.br/downloads/arquivos/LevantamentoGraos.txt"

df_levgraos = pd.read_csv(caminho_arquivo_levgraos, delimiter=';')

df_levgraos = df_levgraos[["ano_agricola","uf","produto","producao_mil_t","area_plantada_mil_ha"]]

df_levgraos['Key2'] = df_levgraos['ano_agricola'].astype(str) + df_levgraos['uf'].astype(str) + df_levgraos['produto'].astype(str) + df_levgraos['producao_mil_t'].astype(str) + df_levgraos['area_plantada_mil_ha'].astype(str)

df_levgraos = df_levgraos[["ano_agricola","uf","produto","producao_mil_t","area_plantada_mil_ha","Key2"]]
df_levgraos['Key2'] = df_levgraos['Key2'].astype(str) 

df_levgraos = df_levgraos.groupby('Key2').agg({
    "ano_agricola":"first",
    "uf":"first",
    "produto":"first",
    "producao_mil_t":"sum",
    "area_plantada_mil_ha":"sum"
}).reset_index()

df_levgraos = df_levgraos[["ano_agricola","uf","produto","producao_mil_t","area_plantada_mil_ha"]]


,ano_agricola,uf,produto,producao_mil_t,area_plantada_mil_ha
0,2017,AM,CAFE,7.0,429.0
1,2017,AM,CAFE,16.0,1008.0
2,2017,AM,CAFE,8.0,543.0
3,2017,BA,CAFE,2380.0,46770.0
4,2017,BA,CAFE,978.0,94871.0
...,...,...,...,...,...
7725,2023,NI,CAFE,89.9,3532.0
7726,2023,PR,CAFE,733.0,27109.0
7727,2023,RJ,CAFE,259.6,10673.0
7728,2023,RO,CAFE,2943.5,64977.0


In [133]:
df_levgraos['safra'].unique()

array(['UNICA          ', '2ª SAFRA       ', '1ª SAFRA       ',
       '3ª SAFRA       '], dtype=object)

In [150]:
pandas_gbq.to_gbq(df_levgraos, 'EC.Dados_Lev_Graos', project_id='fluted-current-384614', if_exists='replace')

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1000.31it/s]
